访问者模式，是**行为型设计模式**之一。访问者模式是一种将数据操作与数据结构分离的设计模式。

访问者模式的基本思想是：  
软件系统中拥有一个由许多对象构成的、比较稳定的对象结构，   
这些对象的类都拥有一个**accept**方法来接收访问者对象的访问。   
访问者是一个接口，拥有一个**visit**方法，这个方法对访问到的对象结构中不同类型的元素作出不同的处理。 在对象结构的一次访问过程中， 会遍历整个对象结构，对每一个元素都实施 accept 方法，在每一个元素的 accept 方法中会调用访问者的 visit 方法，从而是访问者得以处理对象结构的每一个元素， 可以针对对象结构设计不同的访问者类来完成不同的操作，达到区别对待的效果

定义：封装一些作用于某种数据结构中的各元素的操作，它可以在不改变这个数据结构的前提下定义作用于这些元素的操作。

使用场景：   
1） 对象结构比较稳定，当经常需要在此对象结构上定义新的操作   
2） 需要对一个对象结构中的对象进行很多不同的且不相关的操作，而需要避免这些操作“污染”这些对象的类，也不希望在增加新操作时修改这些类。

这种问题在编程领域中很常见，有时候会构建一个由大量不同对象醉成的数据结构。   
例如，要写一个标识数学表达式的程序，那么可能要定义下面这些类：

In [1]:
class Node:
    pass

# 一元操作符类
class UnaryOperator(Node):
    def __init__(self, operand):
        self.operand = operand
    
# 二元操作符类
class BinaryOperator(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right
    
class Add(BinaryOperator):
    pass

class Sub(BinaryOperator):
    pass

class Mul(BinaryOperator):
    pass

class Div(BinaryOperator):
    pass

class Negate(UnaryOperator):
    pass

class Number(Node):
    def __init__(self, value):
        self.value = value

然后利用这些类，构造嵌套数据结构：

In [2]:
# 1+2*（3-4）/5
t1 = Sub(Number(3), Number(4))
t2 = Mul(Number(2), t1)
t3 = Div(t2, Number(5))
t4 = Add(Number(1), t3)

这样写的问题是对于每个表达式，每次都要重新定义一遍，使用访问者模式可以很容易解决这个问题

In [3]:
class NodeVisitor:
    def visit(self, node):
        methodname = 'visit_' + type(node).__name__
        method = getattr(self, methodname, None)
        if method is None:
            method = self.generic_visit
        return method(node)
    
    def generic_visit(self, node):
        raise RuntimeError('No {} method'.format('visit_'+type(node).__name__))

为了使用这个类，可以定义一个类继承并实现各种 `visit_Name()` 方法，其中Name是node类型。例如如果想求表达式的值，可以这么写：

In [4]:
class Evaluator(NodeVisitor):
    def visit_Number(self, node):
        return node.value
    
    def visit_Add(self, node):
        return self.visit(node.left) + self.visit(node.right)
    
    def visit_Sub(self, node):
        return self.visit(node.left) - self.visit(node.right)
    
    def visit_Mul(self, node):
        return self.visit(node.left) * self.visit(node.right)
    
    def visit_Div(self, node):
        return self.visit(node.left) / self.visit(node.right)
    
    def visit_Negate(self, node):
        return -node.operand

In [5]:
e = Evaluator()
e.visit(t4)

0.6

作为一个不同的例子，下面定义一个类在一个栈上面将一个表达式转化为多个操作序列：

In [9]:
class StackCode(NodeVisitor):
    def generate_code(self, node):
        self.instructions = []
        self.visit(node)
        return self.instructions
    
    def visit_Number(self, node):
        self.instructions.append(('PUSH', node.value))
    
    def binop(self, node, instructions):
        self.visit(node.left)
        self.visit(node.right)
        self.instructions.append((instructions,))
    
    def visit_Add(self, node):
        self.binop(node, 'ADD')

    def visit_Sub(self, node):
        self.binop(node, 'SUB')

    def visit_Mul(self, node):
        self.binop(node, 'MUL')

    def visit_Div(self, node):
        self.binop(node, 'DIV')

    def unaryop(self, node, instruction):
        self.visit(node.operand)
        self.instructions.append((instruction,))

    def visit_Negate(self, node):

        self.unaryop(node, 'NEG')

In [10]:
s = StackCode()
s.generate_code(t4)

[('PUSH', 1),
 ('PUSH', 2),
 ('PUSH', 3),
 ('PUSH', 4),
 ('SUB',),
 ('MUL',),
 ('PUSH', 5),
 ('DIV',),
 ('ADD',)]

相对于使用大量的if/else语句，访问者模式的好处就是通过 gettattr() 来获取对应的方法，并利用递归来遍历所有的节点.  
这种技术也是其他语言中switch 或 case 语句的方法。比如：如果你正在写一个HTTP框架，可能学这样写一个请求分发的控制器：

In [11]:
class HTTPHandler:
    def handle(self, request):
        methname = 'do_' + request.request_method
        getattr(self, methname)(request)
        
    def do_GET(self, request):
        pass
    
    def do_POST(self, request):
        pass
    
    def do_HEAD(self, request):
        pass

访问者模式的一个缺点就是严重依赖于递归，如果数据结构嵌套层次太深可能会有问题，有时会超过Python的递归深度限制。  
`sys.getrecursionlimit()`   
此时可以利用生成器或者迭代器来实现非递归遍历运算   
在解析和编译相关的编程中，python本身的ast模块可以看一下